In [11]:
import numpy as np
import matplotlib.pyplot as plt

import pose
from pose_parser import parse_file, detect_perspective


In [12]:
good_videos = [parse_file("dataset/bicep/bicep_good_"+ str(i) +".npy") for i in range(1,7)]
bad_videos = [parse_file("dataset/bicep/bicep_bad_" + str(i) + ".npy") for i in range(1,11)]



Data shape:  (114, 18, 3)
Mean torso:  360.11485467535044
Data shape:  (101, 18, 3)
Mean torso:  361.91618075058716
Data shape:  (140, 18, 3)
Mean torso:  274.49230143795376
Data shape:  (122, 18, 3)
Mean torso:  276.056770970454
Data shape:  (113, 18, 3)
Mean torso:  386.10674117118276
Data shape:  (117, 18, 3)
Mean torso:  388.8324699035529
Data shape:  (108, 18, 3)
Mean torso:  359.69948300318237
Data shape:  (146, 18, 3)
Mean torso:  362.94356251552193
Data shape:  (92, 18, 3)
Mean torso:  364.56482601788053
Data shape:  (120, 18, 3)
Mean torso:  364.55769028003914
Data shape:  (123, 18, 3)
Mean torso:  271.04817985102164
Data shape:  (100, 18, 3)
Mean torso:  274.544553716319
Data shape:  (127, 18, 3)
Mean torso: 276.5870125620258
Data shape:  (102, 18, 3)
Mean torso:  389.4333451516014
Data shape:  (108, 18, 3)
Mean torso:  366.72702499907354
Data shape:  (80, 18, 3)
Mean torso:  382.9545547331215


In [13]:
# old one
for video in good_videos:
    frames = video

    side = detect_perspective(frames)

    if (side == pose.Side.right):
        upper_arm_vecs = np.array([pose.Joint.vector_from_joints(frame.rshoulder, frame.relbow) for frame in frames])
        lower_arm_vecs = np.array([pose.Joint.vector_from_joints(frame.relbow, frame.rwrist) for frame in frames])
    else:
        upper_arm_vecs = np.array([pose.Joint.vector_from_joints(frame.lshoulder, frame.lelbow) for frame in frames])
        lower_arm_vecs = np.array([pose.Joint.vector_from_joints(frame.lelbow, frame.lwrist) for frame in frames])

    # print(lower_arm_vecs)
    print (upper_arm_vecs.shape)
    print (upper_arm_vecs[0])

    #Find the unit vector vec/mag(normalization axis=1)
    upper_arm_vecs = upper_arm_vecs/ np.expand_dims(np.linalg.norm(upper_arm_vecs, axis=1), axis=1)
    lower_arm_vecs = lower_arm_vecs/ np.expand_dims(np.linalg.norm(lower_arm_vecs, axis=1), axis=1)

    #Angle = cos inverse of product of two unit vectors
    # np.multiply(upper_arm_vecs, lower_arm_vecs)
    # np.sum(np.multiply(upper_arm_vecs, lower_arm_vecs), axis=1)
    upper_arm_forearm_angle = np.degrees(np.arccos(np.clip(np.sum(np.multiply(upper_arm_vecs, lower_arm_vecs), axis=1), -1.0, 1.0)))
    print(upper_arm_forearm_angle.shape)



Primary arm: Right
(114, 2)
[0.08156564 0.51118136]
(114,)
Primary arm: Right
(101, 2)
[0.0650786  0.50347846]
(101,)
Primary arm: Right
(140, 2)
[-0.05007426  0.58509838]
(140,)
Primary arm: Right
(122, 2)
[-0.0494862   0.56039922]
(122,)
Primary arm: Left
(113, 2)
[0.02037001 0.52791878]
(113,)
Primary arm: Left
(117, 2)
[-0.03531341  0.58447537]
(117,)


In [14]:
upper_arm_vecs = upper_arm_vecs/ np.expand_dims(np.linalg.norm(upper_arm_vecs, axis=1), axis=1)
lower_arm_vecs = lower_arm_vecs/ np.expand_dims(np.linalg.norm(upper_arm_vecs, axis=1), axis=1)

In [34]:
# Clearer code for angle
for video in good_videos:
    frames = video

    side = detect_perspective(frames)

    #Angles to calculate
    upperarm_forearm_angles = []
    for frame in frames:
        if (side == pose.Side.right):
            upperarm_forearm_joints = [frame.rshoulder, frame.relbow, frame.relbow, frame.rwrist]
        else:
            upperarm_forearm_joints = [frame.lshoulder, frame.lelbow, frame.lelbow, frame.lwrist]
        
        #Pass 4 joints - joint1, joint2 make one vec and joint3 and joint4 make another vec
        upperarm_forearm_angle = pose.Joint.calculate_angle(*upperarm_forearm_joints)
        # print(upperarm_forearm_angle)
        upperarm_forearm_angles.append(upperarm_forearm_angle)
    print(len(upperarm_forearm_angles))
    print(upperarm_forearm_angles[:10])





Primary arm: Right
114
[7.181805962695296, 6.318693492254908, 6.321712578784855, 6.157778476184139, 6.664120851454403, 8.093517037808134, 7.743465327864483, 7.49014760825313, 8.198115773180614, 8.704940075573054]
Primary arm: Right
101
[6.113107148341152, 6.21762171976902, 8.60090636729885, 8.44679588372694, 10.051993112960886, 9.82165617986894, 12.78015299125649, 11.906724328656892, 14.56782941338859, 17.692772092464338]
Primary arm: Right
140
[25.919019399918458, 25.290481717471707, 23.57808198866265, 23.577462317803228, 24.103830893608666, 24.10812376257264, 24.150628520480257, 23.649184140289293, 23.15889389246775, 22.435074541346115]
Primary arm: Right
122
[18.151219376125177, 17.355830185662285, 17.512443741781453, 18.171106753837993, 19.074151609383577, 19.90901281093911, 22.362646019616616, 22.19008735969188, 22.260510547745437, 23.894994910480804]
Primary arm: Left
113
[2.949687907683666, 2.198817674689426, 2.183732384321233, 2.9336743409183565, 4.14873267412005, 7.34169522566